In [54]:
import geopandas as gpd
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

# Paso 1: Dataset Base: IUCN.

In [55]:
# Leemos el dataset, que viene en puntos
# El CRS EPSG:4326 significa latitud/longitud
iucn = gpd.read_file("../data/GeoDataFrame/gdf_species.gpkg").to_crs("EPSG:4326") 

In [56]:
# Seleccionamos las columnas que nos interesan
iucn = iucn[[
    "sci_name",
    "redlistCategory",
    "geometry"
]].copy()

In [57]:
# Creamos una columna con el ID
iucn["iucn_id"] = iucn.index

In [58]:
# Creamos columna con el índice de vulneravilidad:

status_map = {"CR":4, "EN":3, "VU":2, "NT":1}

iucn["vuln"] = (
    iucn["redlistCategory"]
    .map(status_map)
    .fillna(0)
)
iucn

,sci_name,redlistCategory,geometry,iucn_id,vuln
0,Hubbsina turneri,CR,POINT (-101.4795 19.8745),0,4
1,Hubbsina turneri,CR,POINT (-101.7773 19.8256),1,4
2,Hubbsina turneri,CR,POINT (-101.7876 19.8273),2,4
3,Ictalurus mexicanus,VU,POINT (-99.35417 21.98083),3,2
4,Ictalurus mexicanus,VU,POINT (-99.3 22),4,2
...,...,...,...,...,...
70268,Macrobrachium thysi,VU,POINT (-3.00185 5.14407),70268,2
70269,Macrobrachium thysi,VU,POINT (-3.52102 5.49955),70269,2
70270,Macrobrachium thysi,VU,POINT (-3.51293 5.49003),70270,2
70271,Macrobrachium thysi,VU,POINT (-4.12282 5.40212),70271,2


Salida:

iucn = dataset base, ≥60k filas

# Paso 2. NOAA (microplásticos)

In [59]:
# NOAA también son puntos
noaa = gpd.read_file("../data/GeoDataFrame/gdf_microplastics.gpkg").to_crs("EPSG:4326")
noaa

,microplastics_measurement,unit,concentration_class_range,mesh_size_mm,lat,lon,geometry
0,0.000000,pieces/m3,0-0.0005,0.3350,45.280000,-60.290000,POINT (-60.29 45.28)
1,0.002276,pieces/m3,0.0005-0.005,0.3350,40.930000,-70.650000,POINT (-70.65 40.93)
2,0.004320,pieces/m3,0.0005-0.005,0.3350,40.930000,-70.650000,POINT (-70.65 40.93)
3,0.000000,pieces/m3,0-0.0005,0.3350,40.300000,-69.770000,POINT (-69.77 40.3)
4,0.000000,pieces/m3,0-0.0005,0.3350,39.880000,-67.150000,POINT (-67.15 39.88)
...,...,...,...,...,...,...,...
19317,188.300000,pieces kg-1 d.w.,150-200,0.0007,-5.933333,39.360000,POINT (39.36 -5.93333)
19318,155.270000,pieces kg-1 d.w.,150-200,0.0007,-5.500000,39.120000,POINT (39.12 -5.5)
19319,58.070000,pieces kg-1 d.w.,20-150,0.0007,-6.450000,39.466667,POINT (39.46667 -6.45)
19320,210.000000,pieces kg-1 d.w.,>200,0.0007,-6.320000,39.210000,POINT (39.21 -6.32)


Hay filas en las que "microplastics measurement" es NaN y no nos interesan. Nos quedamos solo con las filas con datos medidos

In [60]:
noaa_valid = noaa[noaa["microplastics_measurement"].notna()].copy()
noaa_nan = noaa[noaa["microplastics_measurement"].isna()].copy()
noaa_nan

,microplastics_measurement,unit,concentration_class_range,mesh_size_mm,lat,lon,geometry
13219,NaN,pieces/10 mins,2-40,NaN,27.2049,-97.3645,POINT (-97.3645 27.2049)
13220,NaN,pieces/10 mins,40-200,NaN,27.4147,-97.3016,POINT (-97.3016 27.4147)
13221,NaN,pieces/10 mins,40-200,NaN,27.6057,-97.2077,POINT (-97.2077 27.6057)
13222,NaN,pieces/10 mins,2-40,NaN,26.0983,-97.1623,POINT (-97.1623 26.0983)
13223,NaN,pieces/10 mins,1-2,NaN,27.8322,-97.3784,POINT (-97.3784 27.8322)
...,...,...,...,...,...,...,...
18339,NaN,pieces/10 mins,2-40,NaN,29.3657,-94.8122,POINT (-94.8122 29.3657)
18340,NaN,pieces/10 mins,2-40,NaN,29.3351,-94.7287,POINT (-94.7287 29.3351)
18341,NaN,pieces/10 mins,40-200,NaN,29.3360,-94.7352,POINT (-94.7352 29.336)
18342,NaN,pieces/10 mins,2-40,NaN,32.7114,-96.9773,POINT (-96.9773 32.7114)


Convertimos "noaa_valid" en el nuevo dataset de "noaa"

In [61]:
noaa = noaa_valid[[
    "microplastics_measurement",
    "geometry"
]].copy()

# Renombramos la columna para tener a la vista las unidades de medida
noaa = noaa.rename(columns={
    "microplastics_measurement": "mp_pieces_m3"
})


#### Asignar microplásticos por cercanía

In [62]:
# Pasar a metros
# El CRS EPSG:3857 signiifica proyección métrica
iucn_m = iucn.to_crs("EPSG:3857")
noaa_m = noaa.to_crs("EPSG:3857")

In [63]:
# Nearest spatial join
iucn_noaa = gpd.sjoin_nearest(
    iucn_m,
    noaa_m,
    how="left",
    distance_col="dist_m"
)
iucn_noaa

,sci_name,redlistCategory,geometry,iucn_id,vuln,index_right,mp_pieces_m3,dist_m
0,Hubbsina turneri,CR,POINT (-11296646.266 2258169.64),0,4,13102,705.218618,244627.454273
0,Hubbsina turneri,CR,POINT (-11296646.266 2258169.64),0,4,13101,0.000000,244627.454273
0,Hubbsina turneri,CR,POINT (-11296646.266 2258169.64),0,4,13103,1410.437236,244627.454273
0,Hubbsina turneri,CR,POINT (-11296646.266 2258169.64),0,4,13104,2115.655853,244627.454273
1,Hubbsina turneri,CR,POINT (-11329797.21 2252382.257),1,4,13102,705.218618,237016.489842
...,...,...,...,...,...,...,...,...
70268,Macrobrachium thysi,VU,POINT (-334164.413 573406.106),70268,2,11468,0.038661,959001.335033
70269,Macrobrachium thysi,VU,POINT (-391958.153 613149.338),70269,2,11468,0.038661,929073.410690
70270,Macrobrachium thysi,VU,POINT (-391057.579 612084.684),70270,2,11468,0.038661,929269.937379
70271,Macrobrachium thysi,VU,POINT (-458950.223 602254.212),70271,2,11468,0.038661,866968.071829


Ahora cada especie tiene:

mp_pieces_m3 del NOAA más cercano

dist_m (metros)

In [64]:
# Resolvemos duplicados por microplasticos que estan a la misma distancia, nos quedamos con
iucn_noaa_clean = (
    iucn_noaa
    .sort_values("mp_pieces_m3", ascending=False)
    .drop_duplicates(subset="iucn_id", keep="first")
)
iucn_noaa_clean["iucn_id"].value_counts().max() # Comprobación de que no hay duplicados (debe salir 1)

np.int64(1)

Cuando una especie (iucn_id) tiene varias filas (duplicados), nos quedamos con aquella cuyo valor de mp_pieces_m3 es el más alto.

In [65]:
iucn_noaa = iucn_noaa_clean
iucn_noaa

,sci_name,redlistCategory,geometry,iucn_id,vuln,index_right,mp_pieces_m3,dist_m
26165,Chlorogomphus caloptera,NT,POINT (12071697.089 1339620.367),26165,1,13003,352000.0,629473.690140
26166,Chlorogomphus caloptera,NT,POINT (11991168.569 1283632.005),26166,1,13003,352000.0,551186.291833
26167,Chlorogomphus caloptera,NT,POINT (12137776.338 1368444.022),26167,1,13003,352000.0,696396.278760
28169,Coeliccia mattii,VU,POINT (12067400.156 1333158.119),28169,2,13003,352000.0,625148.154722
28088,Sundaeschna cattienensis,EN,POINT (11935675.803 1263193.8),28088,3,13003,352000.0,498421.861628
...,...,...,...,...,...,...,...,...
21288,Desmophyllum pertusum,VU,POINT (-8874767.179 3306933.223),21288,2,5644,0.0,10135.191377
21287,Desmophyllum pertusum,VU,POINT (-8874767.179 3306907.899),21287,2,5644,0.0,10110.293407
21286,Desmophyllum pertusum,VU,POINT (-8874767.179 3306899.036),21286,2,5644,0.0,10101.579629
21285,Desmophyllum pertusum,VU,POINT (-8874768.292 3306926.892),21285,2,5644,0.0,10128.763530


# Paso 4. Ajustar exposición por distancia

In [66]:
# Pasamos distancia a kilometros
iucn_noaa["distance_km"] = iucn_noaa["dist_m"] / 1000

In [67]:
# Decaimiento exponencial. si el microplástico está a mayor distancia, el impacto va a ser menor
# Evitar asumir que un muestreo lejano afecta igual.

DECAY_KM = 50 

iucn_noaa["mp_effective_m3"] = (
    iucn_noaa["mp_pieces_m3"] *
    np.exp(- iucn_noaa["distance_km"] / DECAY_KM)
)
iucn_noaa

,sci_name,redlistCategory,geometry,iucn_id,vuln,index_right,mp_pieces_m3,dist_m,distance_km,mp_effective_m3
26165,Chlorogomphus caloptera,NT,POINT (12071697.089 1339620.367),26165,1,13003,352000.0,629473.690140,629.473690,1.199509
26166,Chlorogomphus caloptera,NT,POINT (11991168.569 1283632.005),26166,1,13003,352000.0,551186.291833,551.186292,5.741156
26167,Chlorogomphus caloptera,NT,POINT (12137776.338 1368444.022),26167,1,13003,352000.0,696396.278760,696.396279,0.314573
28169,Coeliccia mattii,VU,POINT (12067400.156 1333158.119),28169,2,13003,352000.0,625148.154722,625.148155,1.307901
28088,Sundaeschna cattienensis,EN,POINT (11935675.803 1263193.8),28088,3,13003,352000.0,498421.861628,498.421862,16.493217
...,...,...,...,...,...,...,...,...,...,...
21288,Desmophyllum pertusum,VU,POINT (-8874767.179 3306933.223),21288,2,5644,0.0,10135.191377,10.135191,0.000000
21287,Desmophyllum pertusum,VU,POINT (-8874767.179 3306907.899),21287,2,5644,0.0,10110.293407,10.110293,0.000000
21286,Desmophyllum pertusum,VU,POINT (-8874767.179 3306899.036),21286,2,5644,0.0,10101.579629,10.101580,0.000000
21285,Desmophyllum pertusum,VU,POINT (-8874768.292 3306926.892),21285,2,5644,0.0,10128.763530,10.128764,0.000000


# Paso 5. TOMEx: de concentracion (dosis) a toxicidad

In [68]:
# Cargar TOMEx species
tomex = pd.read_csv("../data/Raw/ToMEx_sp_ml.csv")

In [69]:
# Nos quedeamos con los experimentos en los que se ha observado efecto

tomex_effect = tomex[tomex["Effect"] == "Yes"]

In [70]:
tomex_effect["Unaligned Dose Values"].describe()


count    1.336000e+03
mean     6.166276e+08
std      1.283720e+10
min      4.230910e-04
25%      7.300000e+00
50%      6.250000e+01
75%      1.175069e+04
max      4.060000e+11
Name: Unaligned Dose Values, dtype: float64

In [71]:
## Cálculo de umbral ecotoxicologico. Cuando no existe criterio externo el percentil 50 es el default metodológico

eco_tox_threshold = tomex_effect["Unaligned Dose Values"].median() # Calcula mediana de valores de dosis tóxicas
log_eco_tox_threshold = np.log10(eco_tox_threshold)

Hemos calculado un valor central de dosis tóxica
(el 50 % de los valores están por debajo y el 50 % por encima)
Se utilizará como valor de referencia estadístico para comparar patrones relativos de riesgo

Tomamos la dosis tóxica mediana observada en Tomex como un umbral de referencia y la transformamos a escala logarítmica para su análisis

En ecotoxicología esto se usa para:

clasificar (alto vs bajo riesgo relativo)

comparar especies

analizar patrones

In [72]:
# Convertimos unidades de NOAA (particles/m3 -> particles/ml)
iucn_noaa["mp_particles_ml"] = (
    iucn_noaa["mp_effective_m3"] / 1_000_000
)

Para evitar ver ceros, normal al dividir entre un millón, convertimos los valores a escala logarítmica

In [73]:
iucn_noaa["log_mp_particles_ml"] = np.log10(
    iucn_noaa["mp_particles_ml"] + 1e-12
)

In [74]:
# Presión tóxica relativa
iucn_noaa["log_toxic_pressure"] = (
    iucn_noaa["log_mp_particles_ml"] - log_eco_tox_threshold
)

La presión tóxica relativa se calculó como la diferencia entre la concentración logarítmica de microplásticos y el umbral ecotoxicológico de referencia, representando el número de órdenes de magnitud por los que la exposición se sitúa por encima o por debajo del valor central de toxicidad

Las concentraciones ambientales de microplásticos se sitúan, en general, por debajo de las dosis experimentales reportadas en estudios ecotoxicológicos. Por tanto, los valores de presión tóxica relativa resultan negativos y se interpretan como una medida de proximidad al umbral, no como evidencia de toxicidad directa

Los límites se establecieron de forma exploratoria para diferenciar especies más o menos próximas al umbral ecotoxicológico, dada la escala logarítmica de las concentraciones

In [75]:
def pressure_class(x):
    if x > -3:
        return "high"
    elif x > -6:
        return "medium"
    else:
        return "low"

iucn_noaa["toxic_pressure_class"] = iucn_noaa["log_toxic_pressure"].apply(pressure_class)
iucn_noaa.head()

,sci_name,redlistCategory,geometry,iucn_id,vuln,index_right,mp_pieces_m3,dist_m,distance_km,mp_effective_m3,mp_particles_ml,log_mp_particles_ml,log_toxic_pressure,toxic_pressure_class
26165,Chlorogomphus caloptera,NT,POINT (12071697.089 1339620.367),26165,1,13003,352000.0,629473.690140,629.473690,1.199509,1.199509e-06,-5.920996,-7.716876,low
26166,Chlorogomphus caloptera,NT,POINT (11991168.569 1283632.005),26166,1,13003,352000.0,551186.291833,551.186292,5.741156,5.741156e-06,-5.241001,-7.036881,low
26167,Chlorogomphus caloptera,NT,POINT (12137776.338 1368444.022),26167,1,13003,352000.0,696396.278760,696.396279,0.314573,3.145730e-07,-6.502277,-8.298157,low
28169,Coeliccia mattii,VU,POINT (12067400.156 1333158.119),28169,2,13003,352000.0,625148.154722,625.148155,1.307901,1.307901e-06,-5.883425,-7.679305,low
28088,Sundaeschna cattienensis,EN,POINT (11935675.803 1263193.8),28088,3,13003,352000.0,498421.861628,498.421862,16.493217,1.649322e-05,-4.782695,-6.578575,low


# Paso 6. Cálculo del riesgo ecológico

In [76]:
iucn_noaa["eco_risk_score"] = (
    iucn_noaa["log_toxic_pressure"] *
    iucn_noaa["vuln"]
)

def risk_class(x):
    if x <= -27.6:
        return "low"
    elif x <= -10.9:
        return "medium"
    else:
        return "high"

iucn_noaa["eco_risk_class"] = iucn_noaa["eco_risk_score"].apply(risk_class)
iucn_noaa


,sci_name,redlistCategory,geometry,iucn_id,vuln,index_right,mp_pieces_m3,dist_m,distance_km,mp_effective_m3,mp_particles_ml,log_mp_particles_ml,log_toxic_pressure,toxic_pressure_class,eco_risk_score,eco_risk_class
26165,Chlorogomphus caloptera,NT,POINT (12071697.089 1339620.367),26165,1,13003,352000.0,629473.690140,629.473690,1.199509,1.199509e-06,-5.920996,-7.716876,low,-7.716876,high
26166,Chlorogomphus caloptera,NT,POINT (11991168.569 1283632.005),26166,1,13003,352000.0,551186.291833,551.186292,5.741156,5.741156e-06,-5.241001,-7.036881,low,-7.036881,high
26167,Chlorogomphus caloptera,NT,POINT (12137776.338 1368444.022),26167,1,13003,352000.0,696396.278760,696.396279,0.314573,3.145730e-07,-6.502277,-8.298157,low,-8.298157,high
28169,Coeliccia mattii,VU,POINT (12067400.156 1333158.119),28169,2,13003,352000.0,625148.154722,625.148155,1.307901,1.307901e-06,-5.883425,-7.679305,low,-15.358610,medium
28088,Sundaeschna cattienensis,EN,POINT (11935675.803 1263193.8),28088,3,13003,352000.0,498421.861628,498.421862,16.493217,1.649322e-05,-4.782695,-6.578575,low,-19.735724,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21288,Desmophyllum pertusum,VU,POINT (-8874767.179 3306933.223),21288,2,5644,0.0,10135.191377,10.135191,0.000000,0.000000e+00,-12.000000,-13.795880,low,-27.591760,medium
21287,Desmophyllum pertusum,VU,POINT (-8874767.179 3306907.899),21287,2,5644,0.0,10110.293407,10.110293,0.000000,0.000000e+00,-12.000000,-13.795880,low,-27.591760,medium
21286,Desmophyllum pertusum,VU,POINT (-8874767.179 3306899.036),21286,2,5644,0.0,10101.579629,10.101580,0.000000,0.000000e+00,-12.000000,-13.795880,low,-27.591760,medium
21285,Desmophyllum pertusum,VU,POINT (-8874768.292 3306926.892),21285,2,5644,0.0,10128.763530,10.128764,0.000000,0.000000e+00,-12.000000,-13.795880,low,-27.591760,medium


In [77]:
iucn_noaa["eco_risk_score"].describe()

count    70273.000000
mean       -18.474091
std         10.063624
min        -55.183520
25%        -27.591760
50%        -13.795880
75%        -10.936872
max         -2.628144
Name: eco_risk_score, dtype: float64

In [78]:
iucn_noaa["eco_risk_class"].value_counts()

eco_risk_class
medium    45756
high      17044
low        7473
Name: count, dtype: int64

# Paso 7. Crear Dataset

In [79]:
iucn_noaa.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 70273 entries, 26165 to 21332
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   sci_name              70273 non-null  object  
 1   redlistCategory       70273 non-null  object  
 2   geometry              70273 non-null  geometry
 3   iucn_id               70273 non-null  int64   
 4   vuln                  70273 non-null  int64   
 5   index_right           70273 non-null  int64   
 6   mp_pieces_m3          70273 non-null  float64 
 7   dist_m                70273 non-null  float64 
 8   distance_km           70273 non-null  float64 
 9   mp_effective_m3       70273 non-null  float64 
 10  mp_particles_ml       70273 non-null  float64 
 11  log_mp_particles_ml   70273 non-null  float64 
 12  log_toxic_pressure    70273 non-null  float64 
 13  toxic_pressure_class  70273 non-null  object  
 14  eco_risk_score        70273 non-null  float64 


In [80]:
# Escalamos los valores para que sean más intuitivos de riesgo ecológico
scaler = MinMaxScaler(feature_range=(0, 100))

iucn_noaa["eco_risk_index"] = scaler.fit_transform(
    iucn_noaa[["eco_risk_score"]]
)

In [81]:

def risk_class(x):
    if x <= 20:
        return "very low"
    elif x <= 50:
        return "low"
    elif x <= 80:
        return "medium"
    else:
        return "high"

iucn_noaa["eco_risk_class"] = iucn_noaa["eco_risk_index"].apply(risk_class)
iucn_noaa["eco_risk_class"].value_counts()

eco_risk_class
medium      33695
high        29348
low          6259
very low      971
Name: count, dtype: int64

In [82]:
iucn_noaa.columns

Index(['sci_name', 'redlistCategory', 'geometry', 'iucn_id', 'vuln',
       'index_right', 'mp_pieces_m3', 'dist_m', 'distance_km',
       'mp_effective_m3', 'mp_particles_ml', 'log_mp_particles_ml',
       'log_toxic_pressure', 'toxic_pressure_class', 'eco_risk_score',
       'eco_risk_class', 'eco_risk_index'],
      dtype='object')

In [83]:
new_order = [
    "iucn_id",
    "sci_name",
    "vuln",
    "log_mp_particles_ml",
    "distance_km",
    "log_toxic_pressure",
    "eco_risk_score",
    "eco_risk_index",
    "eco_risk_class",
    "geometry"
]

dataset = iucn_noaa[new_order]


In [84]:
dataset

,iucn_id,sci_name,vuln,log_mp_particles_ml,distance_km,log_toxic_pressure,eco_risk_score,eco_risk_index,eco_risk_class,geometry
26165,26165,Chlorogomphus caloptera,1,-5.920996,629.473690,-7.716876,-7.716876,90.317390,high,POINT (12071697.089 1339620.367)
26166,26166,Chlorogomphus caloptera,1,-5.241001,551.186292,-7.036881,-7.036881,91.611255,high,POINT (11991168.569 1283632.005)
26167,26167,Chlorogomphus caloptera,1,-6.502277,696.396279,-8.298157,-8.298157,89.211354,high,POINT (12137776.338 1368444.022)
28169,28169,Coeliccia mattii,2,-5.883425,625.148155,-7.679305,-15.358610,75.777044,medium,POINT (12067400.156 1333158.119)
28088,28088,Sundaeschna cattienensis,3,-4.782695,498.421862,-6.578575,-19.735724,67.448468,medium,POINT (11935675.803 1263193.8)
...,...,...,...,...,...,...,...,...,...,...
21288,21288,Desmophyllum pertusum,2,-12.000000,10.135191,-13.795880,-27.591760,52.500357,medium,POINT (-8874767.179 3306933.223)
21287,21287,Desmophyllum pertusum,2,-12.000000,10.110293,-13.795880,-27.591760,52.500357,medium,POINT (-8874767.179 3306907.899)
21286,21286,Desmophyllum pertusum,2,-12.000000,10.101580,-13.795880,-27.591760,52.500357,medium,POINT (-8874767.179 3306899.036)
21285,21285,Desmophyllum pertusum,2,-12.000000,10.128764,-13.795880,-27.591760,52.500357,medium,POINT (-8874768.292 3306926.892)


In [85]:
# Guardar

dataset.to_file("../data/dataset.gpkg", layer="ecol_risk", driver="GPKG")
dataset.to_parquet("../data/dataset.parquet")

dataset_csv = dataset.copy()
dataset_csv["geometry"] = dataset_csv.geometry.to_wkt()
dataset_csv.to_csv("../data/dataset_csv.csv", index=False)


/tmp/ipykernel_25235/44233542.py:7: UserWarning: Geometry column does not contain geometry.
  dataset_csv["geometry"] = dataset_csv.geometry.to_wkt()
